# March Madness Predictor Kaggle Competition
This script shows the set-up, preprocessing and data storage for model training.
#### Dependencies

In [1]:
# KenPom Scraper
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

# sklearn 
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
# tensorflow (r0.12)
import tensorflow as tf

## Scrape KenPom website for Team Statistics 
Retrieved during the 2016-2017 competition # find source!

In [2]:
try :
    KP = pd.read_csv('kenpom.csv')
    print("KenPom scrape was completed previously and retrieved from local csv file")
except :

    # Base url, and a lambda func to return url for a given year
    base_url = 'http://kenpom.com/index.php'
    url_year = lambda x: '%s?y=%s' % (base_url, str(x) if x != 2016 else base_url)

    # Years on kenpom's site (could also scrape this and 
    # set as a list if you want to be more dynamic)
    years = range(2002, 2018)

    #%%# Create a method that parses a given year and spits out a raw dataframe
    def import_raw_year(year):
        """
        Imports raw data from a ken pom year into a dataframe
        """
        f = requests.get(url_year(year))
        soup = BeautifulSoup(f.text, "lxml")
        table_html = soup.find_all('table', {'id': 'ratings-table'})

        # Weird issue w/ <thead> in the html
        # Prevents us from just using pd.read_html
        # Let's find all the thead contents and just replace/remove them
        # This allows us to easily put the table row data into a dataframe using panda
        thead = table_html[0].find_all('thead')

        table = table_html[0]
        for x in thead:
            table = str(table).replace(str(x), '')

    #    table = "<table id='ratings-table'>%s</table>" % table
        df = pd.read_html(table)[0]
        df['year'] = year
        return df
    #%%
        # Import all the years into a singular dataframe
    df = None
    for x in years:
        df = pd.concat( (df, import_raw_year(x)), axis=0) \
            if df is not None else import_raw_year(2002)
    #%%
    df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Pyth', 
                   'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
                   'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
                   'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
                   'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank', 'Year']
    #%%
    # Lambda that returns true if given string is a number and a valid seed number (1-16)
    valid_seed = lambda x: True if str(x).replace(' ', '').isdigit() \
                    and int(x) > 0 and int(x) <= 16 else False

    # Use lambda to parse out seed/team
    df['Seed'] = df['Team'].apply(lambda x: x[-2:].replace(' ', '') \
                                  if valid_seed(x[-2:]) else np.nan )

    df['Team'] = df['Team'].apply(lambda x: x[:-2] if valid_seed(x[-2:]) else x)
    #%%       
    # Split W-L column into wins and losses
    df['Wins'] = df['W-L'].apply(lambda x: int(re.sub('-.*', '', x)) )
    df['Losses'] = df['W-L'].apply(lambda x: int(re.sub('.*-', '', x)) )
    df.drop('W-L', inplace=True, axis=1)
    #%%     
    # Reorder columns just cause I'm OCD
    df=df[[ 'Year', 'Rank', 'Team', 'Conference', 'Wins', 'Losses', 'Seed','Pyth', 
                 'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
                 'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
                 'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
                 'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank']]
    #%%
    df.to_csv('kenpom.csv', index=False)
    #%%     
    KP = df.copy()
    del df
    
    print("KenPom Database downloaded and stored as 'kenpom.csv' in local directory.")

        


KenPom scrape was completed previously and retrieved from local csv file


## Scale KenPom data
Scale to min and max of [0,1]

In [3]:
categories = KP.columns
print(categories)

Index(['Year', 'Rank', 'Team', 'Conference', 'Wins', 'Losses', 'Seed', 'Pyth',
       'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank', 'AdjustT',
       'AdjustT Rank', 'Luck', 'Luck Rank', 'SOS Pyth', 'SOS Pyth Rank',
       'SOS OppO', 'SOS OppO Rank', 'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth',
       'NCSOS Pyth Rank'],
      dtype='object')


In [4]:
KP.head()

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,Luck,Luck Rank,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank
0,2002,1,Duke,ACC,31,4,1.0,34.14,118.9,1,...,-0.027,223,10.09,20,107.8,15,97.7,39,7.52,18
1,2002,2,Cincinnati,CUSA,31,4,1.0,30.15,116.0,7,...,0.002,165,7.45,54,105.2,60,97.8,42,5.41,43
2,2002,3,Maryland,ACC,32,4,1.0,29.24,117.2,3,...,0.025,104,10.32,18,107.9,11,97.6,35,5.12,45
3,2002,4,Kansas,B12,31,4,1.0,28.95,116.7,5,...,0.023,109,10.75,13,108.8,7,98.1,49,8.91,9
4,2002,5,Oklahoma,B12,31,5,2.0,26.02,113.0,20,...,0.044,67,9.39,29,107.8,14,98.4,61,2.55,87


In [5]:
drop_bool = [x.find('Rank') != -1 for x in categories]
categories[drop_bool]

Index(['Rank', 'AdjustO Rank', 'AdjustD Rank', 'AdjustT Rank', 'Luck Rank',
       'SOS Pyth Rank', 'SOS OppO Rank', 'SOS OppD Rank', 'NCSOS Pyth Rank'],
      dtype='object')

In [6]:
columns_to_drop = categories[drop_bool].tolist()
columns_to_drop += ['Conference', 'Seed'] 
columns_to_drop

['Rank',
 'AdjustO Rank',
 'AdjustD Rank',
 'AdjustT Rank',
 'Luck Rank',
 'SOS Pyth Rank',
 'SOS OppO Rank',
 'SOS OppD Rank',
 'NCSOS Pyth Rank',
 'Conference',
 'Seed']

In [7]:
KP.drop(columns_to_drop, axis = 1, inplace = True)
print(KP.head())

   Year        Team  Wins  Losses   Pyth  AdjustO  AdjustD  AdjustT   Luck  \
0  2002        Duke    31       4  34.14    118.9     84.7     75.9 -0.027   
1  2002  Cincinnati    31       4  30.15    116.0     85.9     68.6  0.002   
2  2002    Maryland    32       4  29.24    117.2     88.0     74.7  0.025   
3  2002      Kansas    31       4  28.95    116.7     87.8     78.4  0.023   
4  2002    Oklahoma    31       5  26.02    113.0     87.0     68.2  0.044   

   SOS Pyth  SOS OppO  SOS OppD  NCSOS Pyth  
0     10.09     107.8      97.7        7.52  
1      7.45     105.2      97.8        5.41  
2     10.32     107.9      97.6        5.12  
3     10.75     108.8      98.1        8.91  
4      9.39     107.8      98.4        2.55  


In [8]:
X_keys = KP[['Year', 'Team']]
X_data = KP.drop(['Year', 'Team'], axis = 1)

In [9]:
"""Scale X_data mean zero, scale variation based on IQR"""
rob_scaler = RobustScaler()
X_data_scaled = rob_scaler.fit_transform(X_data)


In [10]:
print(X_data.columns)

Index(['Wins', 'Losses', 'Pyth', 'AdjustO', 'AdjustD', 'AdjustT', 'Luck',
       'SOS Pyth', 'SOS OppO', 'SOS OppD', 'NCSOS Pyth'],
      dtype='object')


## Generate dictionary for stats retrieval 

In [11]:
stats_dict = {}
for year in X_keys['Year'].unique() :
    year_dict = {}
    for ix in X_keys.ix[X_keys['Year']==year, :].index :
        stats_list = list(zip(X_data.columns, X_data_scaled[ix,:]))
        team_name = X_keys['Team'].iloc[ix].strip().rstrip('.')
        year_dict.update({team_name : dict(stats_list)})
    
    stats_dict.update({year: year_dict})
print("Test for dictionary generation: {}".format(len(stats_dict) == len(X_keys['Year'].unique())))
    

Test for dictionary generation: True


## Generate dictionary for Team to TeamID conversion

In [12]:
team_ids = pd.read_csv('teams.csv')

In [13]:
id2team_list = list(zip(team_ids['Team_Id'], team_ids['Team_Name']))
team2id_list = list(zip(team_ids['Team_Name'], team_ids['Team_Id']))

In [14]:
team2id_dict = dict(team2id_list)
id2team_dict = dict(id2team_list)

#### Test every Team name key in the stats_dict is in the team_dict
The loop structure below asks the user to search and update team keys as needed for proper alignment between the stats dictionary and the team names provided by Kaggle data documents.

In [24]:

for year in stats_dict :
    for team in stats_dict[year] :
        if team not in team2id_dict : 
            print("="*40)
            print("{} is not in team2id_dict".format(team))
            print("\nPossible matches:")
            for key in team2id_dict :
                if key.find(team[0:5]) != -1 :
                    print(key)
            new_key = input("New key to replace in stats_dict? ('search' if unknown)\n")
            while new_key == 'search' :
                query = input("\nSearch query\n")
                print("\nquery matches:")
                for key in team2id_dict :
                    if key.find(query) != -1 :
                        print(key)
                new_key = input("New key to replace in stats_dict? ('search' if unknown)\n")
            try :
                te_dict = stats_dict[year].pop(team)
                stats_dict[year].update({new_key : te_dict})
                print("\n{} replaced by {} in stats_dict[{}]\n".format(team, new_key, year))
                print("Searching rest of dictionary...")
                #search rest of dictionary to replace the key in other years
                for year in stats_dict :
                    for search_team in stats_dict[year] :
                        if search_team == team :
                            te_dict = stats_dict[year].pop(team)
                            stats_dict[year].update({new_key : te_dict})
                            print("{} replaced by {} in stats_dict[{}]".format(team, new_key, year))
            except :
                print("ERROR in amending {} in {}.  Manual inspection required".format(team, year))

In [26]:
for year in stats_dict :
    print("{} : {} teams".format(year, len(stats_dict[year])))

2016 : 350 teams
2017 : 350 teams
2002 : 326 teams
2003 : 326 teams
2004 : 325 teams
2005 : 329 teams
2006 : 333 teams
2007 : 335 teams
2008 : 340 teams
2009 : 343 teams
2010 : 346 teams
2011 : 344 teams
2012 : 344 teams
2013 : 346 teams
2014 : 350 teams
2015 : 350 teams


In [29]:
import pickle
with open('stats_dictionary.pickle', 'wb') as f :
    pickle.dump(stats_dict, f)


In [32]:
with open('team2id.pickle', 'wb') as f :
    pickle.dump(team2id_dict, f)
with open('id2team.pickle', 'wb') as f :
    pickle.dump(id2team_dict, f)

### Test dictionary retrieval

In [31]:
stats_dict.get(2017).get('Louisville')

{'AdjustD': -1.1304347826086945,
 'AdjustO': 1.617647058823529,
 'AdjustT': -0.093023255813954875,
 'Losses': -1.0,
 'Luck': -0.1764705882352941,
 'NCSOS Pyth': 0.86135181975736574,
 'Pyth': 1.6511768255884127,
 'SOS OppD': -0.37999999999999828,
 'SOS OppO': 2.0000000000000053,
 'SOS Pyth': 1.3772020725388601,
 'Wins': 0.80000000000000004}

## Notes
This notebook contains the steps to download, process and store yearly statistical data from expert Ken Pomeroy's database.  Statistics were stored in a pickled dictionary that can be accessed using team name keys that are consistent with the names provided from the competition data and stored in the team2id and id2team dictionaries.

The next step will be to use these sources for neural network training, then prediction.